In [3]:
from datetime import datetime
import pandas as pd 
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import statsmodels.api as sm
from scipy.optimize import minimize
from scipy import optimize
import warnings
warnings.simplefilter('ignore')

In [ ]:
#original dataset
AER_App_data  =pd.read_csv("AER_App_data.CSV")
#category 5 is the column that is 1 is the app is a game and 0 otherwise
game=AER_App_data['cat5']


In [ ]:
AER_App_data.shape

In [ ]:
AER_App_data

In [ ]:
#Bernoulli model or unconditional probability model
#likelihood function
def bernlik(prob):
    vlog=game*np.log(prob)+(1-game)*np.log(1-prob);
    loglik=np.sum(vlog);
    return loglik

In [ ]:
#the best way to numerically optimize a one dimensional parameter probem is grid search
vgrid= pd.Series(np.arange(0.01, 1, 0.01), name='vgrid')
vloglik = pd.Series(np.zeros(len(vgrid)), name='vloglik')
for j in range(0,len(vgrid)) :
    vloglik[j]=bernlik(vgrid[j])

In [ ]:
plt.plot(vgrid,vloglik,color='blue')
plt.show()

In [ ]:
vgrid

In [ ]:
#rather than visually maximize, we take the point on the grid corresponding to the highest loglikelihood
mloglik=pd.concat([vgrid, vloglik], axis=1)
mloglik.shape


In [ ]:
mloglik=mloglik.sort_values("vloglik", ascending = False)
mloglik.iloc[0,]

In [ ]:
#Alternatively, maximising the loglikelihood function analytically we obtain the empirical mean
game.describe()

In [ ]:
def bernlik(prob):
    vlog=game*np.log(prob)+(1-game)*np.log(1-prob);
    loglik=np.sum(-vlog);
    return loglik

In [ ]:
#rather than a numerical grid search, we can maximise the loglikelihood function numerically 
initParams = [0.5]
results = minimize(bernlik, initParams, method='nelder-mead')
print(results.x)

In [ ]:
#conditional probability models PROBIT
############## Now concentrating only on games which are potentially killer apps
AER_App_data_gameonly=AER_App_data[AER_App_data['cat5']==1]
killer=AER_App_data_gameonly['killerappgros']
score=AER_App_data_gameonly['scoreapp']
killer.describe()

In [ ]:
def probitlik(beta):
    beta0=beta[0]
    beta1=beta[1]
    vlog=killer*np.log(norm.cdf(beta0+beta1*score))+(1-killer)*np.log(1-norm.cdf(beta0+beta1*score));
    probit=sum(-vlog);
    return probit

In [ ]:
n=len(killer)
initParams = [0.1,0.1]
results = minimize(probitlik, initParams, method='nelder-mead')
print(results.x)


In [ ]:
########conditional probability models LOGIT

def logitlik(beta):
    beta0=beta[0]
    beta1=beta[1]
    vlog=killer*np.log(1/(1+np.exp(-beta0-beta1*score)))+(1-killer)*np.log(1-1/(1+np.exp(-beta0-beta1*score)))
    probit=sum(-vlog)
    return probit

In [ ]:
n=len(killer)
initParams = [0.1,0.1]
results = minimize(logitlik, initParams, method='nelder-mead')
print(results.x)



In [ ]:
######
def probitlik_restrict(beta0):
    vlog=killer*np.log(norm.cdf(beta0))+(1-killer)*np.log(1-norm.cdf(beta0))
    probit=sum(-vlog);
    return probit

In [ ]:
n=len(killer)
initParams = [0.1]
results = minimize(probitlik_restrict, initParams, method='nelder-mead')
print(results.x)


In [ ]:
#estimation of the models with the glm function

import statsmodels.api as sm
score=sm.add_constant(score)
probitmodel2 = sm.GLM(killer, score, family=sm.families.Binomial(link=sm.families.links.probit)) 
probitmodel2 =probitmodel2.fit()
print(probitmodel2.summary())

In [ ]:
logitmodel = sm.GLM(killer, score, family=sm.families.Binomial(link=sm.families.links.logit)) 
logitmodel =logitmodel.fit()
print(logitmodel.summary())

In [ ]:
logitmodel2 = sm.GLM(killer, score, family=sm.families.Binomial()) 
logitmodel2=logitmodel2.fit()
print(logitmodel2.summary())

In [ ]:
(0.5261-1)/0.033
